In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import zgoubidoo
from zgoubidoo.commands import *
from zgoubidoo import ureg as _

In [ ]:
kin = zgoubidoo.Kinematics(230 * _.MeV)
kin

In [ ]:
distribution = np.array([
    [0.0, 0.0, 0.0, 0.0],  # Y T Z P == X X' Y Y'
    [1.0, 0.0, 0.0, 0.0],
    [-1.0, 0.0, 0.0, 0.0],
])
beam_for_tracks = BeamInputDistribution('BEAM', kinematics=kin, distribution=distribution)
beam_for_tracks

In [ ]:
beam_for_twiss = BeamTwiss('BEAM', kinematics=kin)
beam_for_twiss

In [ ]:
zi = zgoubidoo.Input(name='TEST', line=[
    beam_for_tracks,
    Proton(),
    Quadrupole('Q1', XL=5*_.cm, B0=-3*_.tesla),
    Drift(XL=20*_.cm),
    Quadrupole('Q2', XL=5*_.cm, B0=3*_.tesla),
    Drift(XL=20*_.cm),
    Quadrupole('Q3', XL=5*_.cm, B0=-3*_.tesla),
    Drift(XL=20*_.cm),
    Quadrupole('Q4', XL=5*_.cm, B0=3*_.tesla),
    Drift(XL=20*_.cm),
    Marker(),
])
zi.IL = 2
zi.survey(with_reference_trajectory=True, reference_kinematics=kin);

In [ ]:
zr = zgoubidoo.Zgoubi()(zi).collect()

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist(
    layout={
        'xaxis' : {'title': 'X (m)', 'mirror': True, 'linecolor': 'black', 'linewidth': 1},
        'yaxis' : {'title': 'Y (m)', 'mirror': True, 'linecolor': 'black', 'linewidth': 1, 'exponentformat': 'power', },
        'legend': {'bordercolor': '#888', 
                   'borderwidth': 1, 
                   'xanchor': 'left', 
                   'x': 0.1, 
                   'yanchor': 'top', 
                   'y': 0.98, 
                   'orientation': "h"
                  },
    }
)
artist.plot_beamline(beamline=zi, opacity=0.8)

for y0 in zr.tracks['Yo'].unique():
    t = zr.tracks_global.query(f"Yo == {y0}")
    artist.scatter(
            x=t['XG'],
            y=t['YG'],
            marker={'color': 'blue', 'symbol': 303, 'size': .1},
            name='Nominal',
            line={'width': 2, },
            mode='markers+lines',
        )

artist.render()

In [ ]:
zgoubidoo.twiss.compute_periodic_twiss(zr.step_by_step_transfer_matrix)

In [ ]:
zi.replace("BEAM", beam_for_twiss)
zr = zgoubidoo.Zgoubi()(zi).collect()
tw = zgoubidoo.twiss.compute_twiss(matrix=zr.step_by_step_transfer_matrix, with_phase_unrolling=True)

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist(
    layout={
        'xaxis' : {'title': 'S (m)', 'mirror': True, 'linecolor': 'black', 'linewidth': 1},
        'yaxis' : {'title': 'Y (m)', 'mirror': True, 'linecolor': 'black', 'linewidth': 1, 'exponentformat': 'power', },
        'legend': {'bordercolor': '#888', 
                   'borderwidth': 1, 
                   'xanchor': 'left', 
                   'x': 0.1, 
                   'yanchor': 'top', 
                   'y': 0.98, 
                   'orientation': "h"
                  },
    }
)
artist.plot_cartouche(beamline=zi)

artist.scatter(
        x=tw['S'],
        y=tw['BETA11'],
        marker={'color': 'blue', 'symbol': 303, 'size': 3},
        line={'width': 1, },
        name='Horizontal optics (1 sigma)',
        mode='markers+lines',
    )

artist.scatter(
        x=tw['S'],
        y=tw['BETA22'],
        marker={'color': 'red', 'symbol': 303, 'size': 3},
        line={'width': 1, },
        name='Horizontal optics (1 sigma)',
        mode='markers+lines',
    )

artist.add_secondary_axis()

artist.scatter(
        x=tw['S'],
        y=tw['DISP1'],
        marker={'color': 'green', 'symbol': 303, 'size': 3},
        line={'width': 1, },
        name='Dispersion',
        mode='markers+lines',
        yaxis='y2',
    )

artist.render()

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist(
    layout={
        'xaxis' : {'title': 'S (m)', 'mirror': True, 'linecolor': 'black', 'linewidth': 1},
        'yaxis' : {'title': 'Y (m)', 'mirror': True, 'linecolor': 'black', 'linewidth': 1, 'exponentformat': 'power', },
        'legend': {'bordercolor': '#888', 
                   'borderwidth': 1, 
                   'xanchor': 'left', 
                   'x': 0.1, 
                   'yanchor': 'top', 
                   'y': 0.98, 
                   'orientation': "h"
                  },
    }
)
artist.plot_cartouche(beamline=zi)

artist.scatter(
        x=tw['S'],
        y=tw['ALPHA11'],
        marker={'color': 'blue', 'symbol': 303, 'size': 3},
        line={'width': 1, },
        name='Horizontal optics (1 sigma)',
        mode='markers+lines',
    )

artist.scatter(
        x=tw['S'],
        y=tw['ALPHA22'],
        marker={'color': 'red', 'symbol': 303, 'size': 3},
        line={'width': 1, },
        name='Horizontal optics (1 sigma)',
        mode='markers+lines',
    )

artist.render()

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist(
    layout={
        'xaxis' : {'title': 'S (m)', 'mirror': True, 'linecolor': 'black', 'linewidth': 1},
        'yaxis' : {'title': 'Y (m)', 'mirror': True, 'linecolor': 'black', 'linewidth': 1, 'exponentformat': 'power', },
        'legend': {'bordercolor': '#888', 
                   'borderwidth': 1, 
                   'xanchor': 'left', 
                   'x': 0.1, 
                   'yanchor': 'top', 
                   'y': 0.98, 
                   'orientation': "h"
                  },
    }
)
artist.plot_cartouche(beamline=zi)

artist.scatter(
        x=tw['S'],
        y=tw['GAMMA11'],
        marker={'color': 'blue', 'symbol': 303, 'size': 3},
        line={'width': 1, },
        name='Horizontal optics (1 sigma)',
        mode='markers+lines',
    )

artist.scatter(
        x=tw['S'],
        y=tw['GAMMA22'],
        marker={'color': 'red', 'symbol': 303, 'size': 3},
        line={'width': 1, },
        name='Horizontal optics (1 sigma)',
        mode='markers+lines',
    )

artist.render()

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist(
    layout={
        'xaxis' : {'title': 'S (m)', 'mirror': True, 'linecolor': 'black', 'linewidth': 1},
        'yaxis' : {'title': 'Y (m)', 'mirror': True, 'linecolor': 'black', 'linewidth': 1, 'exponentformat': 'power', },
        'legend': {'bordercolor': '#888', 
                   'borderwidth': 1, 
                   'xanchor': 'left', 
                   'x': 0.1, 
                   'yanchor': 'top', 
                   'y': 0.98, 
                   'orientation': "h"
                  },
    }
)
artist.plot_cartouche(beamline=zi)

artist.scatter(
        x=tw['S'],
        y=tw['MU1'],
        marker={'color': 'blue', 'symbol': 303, 'size': 3},
        line={'width': 1, },
        name='Horizontal optics (1 sigma)',
        mode='lines',
    )

artist.scatter(
        x=tw['S'],
        y=tw['MU1U'],
        marker={'color': 'blue', 'symbol': 303, 'size': 3},
        line={'width': 1, },
        name='Horizontal optics (1 sigma)',
        mode='markers',
    )

artist.scatter(
        x=tw['S'],
        y=tw['MU2'],
        marker={'color': 'red', 'symbol': 303, 'size': 3},
        line={'width': 1, },
        name='Horizontal optics (1 sigma)',
        mode='markers+lines',
    )

artist.render()